In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing

In [4]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\SHIVAM
[nltk_data]     SINGH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission_data = pd.read_csv("sample_submission_UVKGLZE.csv")

In [6]:
text_train=train['TITLE']+train['ABSTRACT']
text_test=test['TITLE']+test['ABSTRACT']
text=pd.concat([text_train,text_test])


In [7]:
# Replace numbers with 'numbr'
processed_text =text.str.replace(r'\d+(\.\d+)?', 'numbr')

# Remove punctuation
processed_text = processed_text.str.replace(r'[^\w\d\s]', ' ')

# Replace whitespace between terms with a single space
processed_text = processed_text.str.replace(r'\s+', ' ')

# Remove leading and trailing whitespace
processed_text = processed_text.str.replace(r'^\s+|\s+?$', '')

# change words to lower case - Hello, HELLO, hello are all the same word
processed_text = processed_text.str.lower()

In [8]:
from nltk.corpus import stopwords

# remove stop words from text messages

stop_words = set(stopwords.words('english'))

processed_text = processed_text.apply(lambda x: ' '.join(
    term for term in x.split() if term not in stop_words))

In [9]:
# Modify words with Word Net Lemmatizer
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

processed_text=processed_text.apply(lambda x: ' '.join(
    wordnet_lemmatizer.lemmatize(term) for term in x.split()))

In [10]:
# Remove word stems using a Porter stemmer
ps = nltk.PorterStemmer()

processed_text = processed_text.apply(lambda x: ' '.join(
    ps.stem(term) for term in x.split()))

In [11]:
processed_text_train=processed_text[:20972]
processed_text_test=processed_text[20972:]

In [12]:
total_size=train.shape[0]
train_size=int(0.8*total_size)

X_train=processed_text_train.head(train_size)
X_test=processed_text_train.tail(total_size-train_size)

y = train.iloc[:, 3:]
y_train=y[0:train_size]
y_test=y[train_size:total_size]

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(min_df=1, smooth_idf=True, norm="l2",
                          tokenizer=lambda x: x.split(),sublinear_tf=True, ngram_range=(1,3))

X_train_multilabel=vectorizer.fit_transform(X_train)
X_test_multilabel=vectorizer.transform(X_test)

In [14]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score

classifier=OneVsRestClassifier(LinearSVC(penalty="l2",loss='hinge'), n_jobs=-1)
classifier.fit(X_train_multilabel,y_train)
predictions=classifier.predict(X_test_multilabel)

print(f1_score(y_test,predictions, average='micro'))

0.8326906799586193


In [15]:
test_multilabel=vectorizer.transform(processed_text_test)
predictions=classifier.predict(test_multilabel)

In [18]:
submission=pd.DataFrame(predictions, columns=['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance'])
submission=pd.concat([submission_data['ID'],submission],axis=1)
submission.to_csv("topic.csv", index=False)
print(submission.shape)
submission.head()

(8989, 7)


,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,0,0,0,0


In [19]:
from sklearn.linear_model import SGDClassifier
from sklearn.multioutput import MultiOutputClassifier

clf = SGDClassifier(epsilon=0.01, max_iter=3000,class_weight='balanced')
multi_target_classifier = MultiOutputClassifier(clf, n_jobs=-1)

In [20]:
multi_target_classifier.fit(X_train_multilabel,y_train)

MultiOutputClassifier(estimator=SGDClassifier(class_weight='balanced',
                                              epsilon=0.01, max_iter=3000),
                      n_jobs=-1)

In [24]:
y_pred=multi_target_classifier.predict(X_test_multilabel)

print(f1_score(y_test,y_pred, average='micro'))

0.8277068392513557


In [22]:
from lightgbm import LGBMClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier

clf = LGBMClassifier(class_weight='balanced')
model = MultiOutputClassifier(clf)

In [23]:
model.fit(X_train_multilabel,y_train)

MultiOutputClassifier(estimator=LGBMClassifier(class_weight='balanced'))

In [ ]:
preds=model.predict(X_test_multilabel)

print(f1_score(y_test,preds, average='micro'))